In [1]:
import os

In [2]:
%pwd

'c:\\Users\\dzmitry\\Desktop\\data_science_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\dzmitry\\Desktop\\data_science_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: float
    learning_rate: float
    max_depth: float
    min_child_weight: float
    subsample: float
    colsample_bytree: float
    gamma: float
    reg_alpha: float
    reg_lambda: float
    scale_pos_weight: float
    objective: str
    eval_metric: str
    random_state: int 
    target_column: str

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            learning_rate = params.learning_rate,
            max_depth=params.max_depth,
            min_child_weight=params.min_child_weight,
            subsample=params.subsample,
            colsample_bytree=params.colsample_bytree,
            gamma=params.gamma,
            reg_alpha=params.reg_alpha,
            reg_lambda=params.reg_lambda,
            scale_pos_weight=params.scale_pos_weight,
            objective=params.objective,
            eval_metric=params.eval_metric,
            random_state=params.random_state,
            target_column = schema.name)

        return model_trainer_config

In [11]:
import pandas as pd
import os
from src.datascience import logger
from xgboost import XGBClassifier
import joblib

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = XGBClassifier( n_estimators=self.config.n_estimators, learning_rate=self.config.learning_rate,max_depth=self.config.max_depth,min_child_weight=self.config.min_child_weight,
                            subsample=self.config.subsample,colsample_bytree=self.config.colsample_bytree,gamma=self.config.gamma,reg_alpha=self.config.reg_alpha,reg_lambda=self.config.reg_lambda,
                            scale_pos_weight=self.config.scale_pos_weight,objective=self.config.objective,eval_metric=self.config.eval_metric,random_state=self.config.random_state)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


    

In [14]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2026-01-04 18:40:45,903: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-04 18:40:45,907: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-04 18:40:45,911: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-01-04 18:40:45,915: INFO: common: created directory at: artifacts]
[2026-01-04 18:40:45,917: INFO: common: created directory at: artifacts/model_trainer]
